In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['LLAMA_INDEX_TOKEN'] = os.getenv('LLAMA_INDEX_TOKEN')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from llama_parse import LlamaParse
llama_cloud_api_key = os.environ['LLAMA_INDEX_TOKEN']

In [5]:
documents = LlamaParse(
    api_key=llama_cloud_api_key,
    result_type="markdown",
    content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
).load_data(
    "data/resume.pdf",
)

Started parsing the file under job_id cdd64ddd-b30f-4d5d-9d32-c57b6a71d042


In [6]:
for document in documents:
    print(document.text)

# Aman Lonare

Email: amanlonare95@gmail.com

Phone: +81‑70‑9018‑5707

Location: Tokyo, Japan

# PROFESSIONAL EXPERIENCE

# Hitachi R&D, Center for Technology Innovation

Tokyo, JPN

Research Software Engineering, Services Computing Research Dept.
Jan 2021 – Present

- Developed a tool in Python for assisting in the implementation of CQRS and Event Sourcing design patterns for business users
- Designed and performed evaluation of Core Banking application using AxonIQ Framework with Prometheus and Grafana
- Reduced the overall development time by 15% using Domain Driven Design (DDD) principles of software development process
- Facilitated the implementation of event driven architecture in system and integrated Machine Learning solutions with the tool
- Proposed a plan to integrate the developed tool in Hitachi’s indigenous framework for microservices application development

Patent: “System and method to assist the modelling of CQRS and Event Sourcing based application”, In preparation


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

In [8]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [9]:
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
)

In [10]:
from llama_index.llms.groq import Groq

In [11]:
groq_api_key = os.environ['GROQ_API_KEY']
llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
llm

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x311bf4cd0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x13900ce00>, completion_to_prompt=<function default_completion_to_prompt at 0x139817240>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama-3.2-3b-preview', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, reasoning_effort=None, modalities=None, audio_config=None, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [12]:
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=5,
)
response = query_engine.query(
    "What is this person name and what was their most recent job"
)


In [13]:
storage_dir = "./storage"
index.storage_context.persist(
    persist_dir=storage_dir,
)

In [14]:
from llama_index.core import StorageContext, load_index_from_storage

if os.path.exists(storage_dir):
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    restored_index = load_index_from_storage(storage_context)
else:
    print("Index not found on disk")

In [15]:
response = restored_index.as_query_engine(llm=llm, similarity_top_k=5,).query(
    "What is this person name and what was their most recent job"
)
print(response)

This person's name is Aman Lonare. Their most recent job is Research Software Engineering at Hitachi R&D, Center for Technology Innovation, where they have been working since January 2021.


In [16]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

## The below function will be provided to the llm which will assess the docstring, name and metadata of the function and decides if it useful or not

In [17]:
def query_resume(q: str):
    """Answer questions about a specific resume."""
    # we are using query engine we created from above
    response = query_engine.query(f"This is a question about the specific resume: {q}")
    return response.response


## We will turn the method into a tool using FunctionTool

In [18]:

resume_tool = FunctionTool.from_defaults(
    fn=query_resume,
)

## We will create a function calling agent now

In [19]:
agent = FunctionCallingAgent.from_tools(
    tools=[resume_tool], # just one in our case
    llm=llm,
    verbose=True,
)

In [20]:
response = agent.chat("How many years of experience does this person have? Give me exact number of years as output")
print(response)

> Running step d5845877-1268-42f0-b503-68553da30205. Step input: How many years of experience does this person have? Give me exact number of years as output
Added user message to memory: How many years of experience does this person have? Give me exact number of years as output
=== Calling Function ===
Calling function: query_resume with args: {"q": "years of experience"}
=== Function Output ===
The individual has a total of around 7 years of experience in the field of software engineering and technology development.
> Running step 2ea69fd1-075e-4b17-b9bb-315bdd813761. Step input: None
=== Calling Function ===
Calling function: query_resume with args: {"q": "number of years of experience"}
=== Function Output ===
The individual has around 7 years of experience in the field of software engineering and research, with a significant portion of that experience being in Japan.
> Running step cf5ca5f2-1acc-4695-a062-efa667930c3b. Step input: None
=== Calling Function ===
Calling function: que

In [21]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

In [22]:
class QueryEvent(Event):
    """Custom event to query the resume."""
    query: str

## RAG Workflow

In [23]:
class RAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
    query_engine = VectorStoreIndex

    # first step will be setup
    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> QueryEvent:
        
        if not ev.resume_file:
            raise ValueError("Resume file not provided")
        
        # define an llm to work with
        self.llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)

        # ingest data and set up the query engine
        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse the data if storage index is not created
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=Settings.embed_model,
            )
            # persist the index
            index.storage_context.persist(
                persist_dir=self.storage_dir,
            )
        
        # either way, create query engine
        self.query_engine = index.as_query_engine(
            llm=self.llm,
            similarity_top_k=5,
        )

        # fire off the query event defined
        return QueryEvent(query=ev.query)
    
    # second step will be to ask a question and return a result immediately
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        # query the resume
        response = self.query_engine.query(f"This is the question about the specific resume: {ev.query}")
        # return the result
        return StopEvent(result=response.response)

In [24]:
workflow = RAGWorkflow(timeout=60, verbose=False)
result = await workflow.run(
    resume_file="data/resume.pdf",
    query="Where is the first place the person worked?",
)
print(result)

Hitachi R&D, Center for Technology Innovation in Tokyo, Japan.


## Form Parsing

In [25]:
parser = LlamaParse(
    api_key=llama_cloud_api_key,
    content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
    formatting_instruction="Return a bulleted list of the fields ONLY"
)

In [26]:
result = parser.load_data(
    file_path="data/application_form.pdf",
)[0]

Started parsing the file under job_id 0fdb7da0-fb97-4a5a-a4c7-0804a879682b


In [27]:
print(result.text)

First Name:
Last Name:
Email:
Phone:
LinkedIn:
Project Portfolio:
Degree:
Done Post Graduation?:
Post Graduation Date:
Graduation Date:
Current Job Title:
Current Employer:
Technical Skills:
Describe why the candidate is a good fit for this position:
Do you have 5 years of experience in Python?:


In [28]:
llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
raw_json = llm.complete(
    f"""
    This is a parsed from a job application form.
    Convert it into a JSON object containing only the list of fields
    to be filled in, in the form {{"fields": [...]}}.
    <form>{result.text}</form>
    Return JSON only, no markdown please. 
    Clean the text by removing '\n' or new line if there are any.
    Do not add the type and just return the field name as a list
    """
)

In [29]:
raw_json

CompletionResponse(text='{"fields": ["First Name", "Last Name", "Email", "Phone", "LinkedIn", "Project Portfolio", "Degree", "Done Post Graduation?", "Post Graduation Date", "Graduation Date", "Current Job Title", "Current Employer", "Technical Skills", "Describe why the candidate is a good fit for this position", "Do you have 5 years of experience in Python?"]}', additional_kwargs={'prompt_tokens': 190, 'completion_tokens': 84, 'total_tokens': 274}, raw=ChatCompletion(id='chatcmpl-7b6ae147-485b-4db6-93c8-7eeefc0b13c1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"fields": ["First Name", "Last Name", "Email", "Phone", "LinkedIn", "Project Portfolio", "Degree", "Done Post Graduation?", "Post Graduation Date", "Graduation Date", "Current Job Title", "Current Employer", "Technical Skills", "Describe why the candidate is a good fit for this position", "Do you have 5 years of experience in Python?"]}', refusal=None, role='assistant',

In [30]:
import json
fields = json.loads(raw_json.text)['fields']

for field in fields:
    print(field)


First Name
Last Name
Email
Phone
LinkedIn
Project Portfolio
Degree
Done Post Graduation?
Post Graduation Date
Graduation Date
Current Job Title
Current Employer
Technical Skills
Describe why the candidate is a good fit for this position
Do you have 5 years of experience in Python?


In [31]:
class ParseFormEvent(Event):
    """Custom event to parse the form."""
    application_form: str

class QueryEvent(Event):
    """Custom event to query the resume."""
    query: str

class ResponseEvent(Event):
    """Custom event to return the response."""
    response: str

## RAG Workflow

In [32]:
class FormParserRAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
    query_engine = VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")
        
        # define an llm to work with
        self.llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)

        # ingest data and set up the query engine
        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse the data if storage index is not created
            parser = LlamaParse(
                api_key=llama_cloud_api_key,
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=Settings.embed_model,
            )
            # persist the index
            index.storage_context.persist(
                persist_dir=self.storage_dir,
            )
        
        # either way, create query engine
        self.query_engine = index.as_query_engine(
            llm=self.llm,
            similarity_top_k=5,
        )

        # fire off the query event defined
        return ParseFormEvent(application_form=ev.application_form)
    
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        
        parser = LlamaParse(
        api_key=llama_cloud_api_key,
        content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
        formatting_instruction="Return a bulleted list of the fields ONLY"
        )

        result = parser.load_data(
            file_path="data/application_form.pdf",
        )[0]

        raw_json = self.llm.complete(
            f"""
            This is a parsed from a job application form.
            Convert it into a JSON object containing only the list of fields
            to be filled in, in the form {{"fields": [...]}}.
            <form>{result.text}</form>
            Return JSON only, no markdown please. 
            Clean the text by removing '\n' or new line if there are any.
            Do not add the type and just return the field name as a list
            """
        )

        fields = json.loads(raw_json.text)['fields']

        for field in fields:
            ctx.send_event(
                QueryEvent(
                    field=field,
                    query=f"How would you answer the question about the candidate? Give me a succinct answer for each field if there is no description like first name etc.",
                    )
            )
    
        await ctx.set("total fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        # query the resume
        response = self.query_engine.query(f"This is the question about the specific resume: {ev.query}")
        # return the result
        return ResponseEvent(field=ev.field, response=response.response)
    
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        total_fields = await ctx.get("total fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)

        if responses is None:
            return None
        
        responseList = "\n".join("Field: "+ r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in a job application form and the responses to questions about those fields from 
            a candidate's resume. Combine the two into a list of field and succinct, factual answers to fill in those fields. Do not
            output 'Field' and 'Response' but fill in the values of them in the format Field: Response.
            <responses>
            {responseList}
            </responses>
            """
        )

        return StopEvent(result=result)
        

In [33]:
workflow_form = FormParserRAGWorkflow(timeout=60, verbose=False)
result = await workflow_form.run(
    resume_file="data/resume.pdf",
    application_form="data/application_form.pdf",
)

Started parsing the file under job_id 99bbb4aa-1ec7-4733-b975-4cd58457c0c7


In [34]:
print(result)

Here is the combined list of fields and succinct, factual answers:

Email: amanlonare95@gmail.com
Phone: +81-70-9018-5707
Location: Tokyo, Japan

Professional Experience:
- Research Software Engineering: Developed a tool for CQRS and Event Sourcing design patterns, designed evaluation of Core Banking application, reduced development time by 15%, facilitated event-driven architecture, and proposed tool integration.
- Project Research Associate: Developed models for detecting onion infection, formulated a nationwide course on Digital Technology for Smart Agriculture, and achieved 95% accuracy.
- Software Developer Intern: Developed functional prototypes for Android application, enabled data visualization using Google Analytics.

Projects:
- Web-based Decision Support System: Developed a web portal for crop monitoring, achieved 78% accuracy in predicting crop yields, and reported increased MSP and reduced spoilage.
- Village Level Identification of Sugarcane Crop: Published a paper on usi

## Human in the Loop

In [35]:
from llama_index.core.workflow import HumanResponseEvent, InputRequiredEvent

In [36]:
class ParseFormEvent(Event):
    """Custom event to parse the form."""
    application_form: str

class QueryEvent(Event):
    """Custom event to query the resume."""
    query: str
    field: str

class ResponseEvent(Event):
    """Custom event to return the response."""
    response: str

class FeedbackEvent(Event):
    """Custom event to return the feedback."""
    feedback: str

class GenerateQuestionsEvent(Event):
    """Custom event to generate questions."""
    pass


In [ ]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm = Groq(model="llama-3.1-8b-instant", api_key=groq_api_key)
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = Groq(model="llama-3.1-8b-instant", api_key=groq_api_key)

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=
                                                           self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=Settings.embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # form parsing
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> GenerateQuestionsEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown.")
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # generate questions
    @step
    async def generate_questions(self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            if hasattr(ev,"feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """
            
            ctx.send_event(QueryEvent(
                field=field,
                query=question
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return
        
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

  
    # Get feedback from the human
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # save the result for later
        await ctx.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )

    # Accept the feedback when a HumanResponseEvent fires
    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)


In [43]:
w = RAGWorkflow(timeout=600, verbose=False)
handler = w.run(
    resume_file="data/resume.pdf",
    application_form="data/application_form.pdf"
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)
        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response
            )
        )

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Started parsing the file under job_id 6cdb15db-cf46-452f-b1c3-3cae30a87041
We've filled in your form! Here are the results:

Here's a list of fields and succinct, factual answers to fill in those fields:

1. **First Name**: Aman
2. **Last Name**: Lonare
3. **Email**: amanlonare95@gmail.com
4. **Phone**: +81-70-9018-5707
5. **LinkedIn**: No information available
6. **Project Portfolio**: 
   - Developed a Web-based Decision Support System (DSS) for Crop Monitoring using remote sensing and open-source satellite images, achieving 78% accuracy and F1-score of 0.8 in predicting crop yields.
   - Created a tool in Python for assisting in the implementation of CQRS and Event Sourcing design patterns for business users.
   - Facilitated the implementation of event-driven architecture in a system and integrated Machine Learning solutions with the tool.
7. **Degree**: Post Graduation from Indian Institute of Technology Bombay, with a specialization in Technology and Development
8. **Done Post Gr